In [11]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import os,sys
from PIL import Image
import random
import albumentations as albu
import cv2
import helper
import pandas as pd 
import h5py
import keras
from keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading the dataset

In [2]:
# Loaded a set of images
root_dir = "Datasets/training/"

n = 100

imgs = []
gt_imgs = []

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
print("Loading " + str(n) + " images")
imgs = [helper.load_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [helper.load_image(gt_dir + files[i]) for i in range(n)]

Loading 100 images
Loading 100 images


# Data pre-processing

In [ ]:
#Standardize 
#standardize(imgs)

#convert all imgs to uint8 type 
uint8_imgs = [None] * len(imgs)
for i, image in enumerate(imgs): 
    uint8_img = helper.img_float_to_uint8(image)
    uint8_imgs[i] = uint8_img

## Data augmentation 

In [ ]:
"Types of augmentation"
random.seed(42)

light = albu.Compose([albu.RandomBrightnessContrast(p=1),albu.RandomGamma(p=1), albu.CLAHE(p=1)], p=1)

medium = albu.Compose([albu.CLAHE(p=1), albu.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50, p=1)], p=1)

strong = albu.Compose([albu.ChannelShuffle(p=1)], p=1)

In [ ]:
def augment_imgs(aug, original_set,ground_set, plot = False): 
    
    new =[None] * len(original_set)
    gt = [None] * len(ground_set)
    
    final = original_set.copy()
    gt_final = ground_set.copy()
    for i, image in enumerate(original_set):
        new[i] = augment_and_show(aug, image, show = plot)
        gt[i] = ground_set[i]
    
    final.extend(new)
    gt_final.extend(gt)

    return final, gt_final

In [3]:
def extract_patches(sat, bk, patch_size = 16):
    
    img_patches = [helper.img_crop(sat[i], patch_size, patch_size) for i in range(len(sat))]
    gt_patches = [helper.img_crop(bk[i], patch_size, patch_size) for i in range(len(bk))]
    
    # Linearize list of patches
    img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
    gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    gt_patches = helper.patches_labelization(gt_patches)

    return img_patches, gt_patches 

In [4]:
#Grid shuffle 


## Original data

In [5]:
img_patches, gt_patches = extract_patches(imgs, gt_imgs)
y = to_categorical(gt_patches) #for categorical_crossentropy

## Light augmentation data 

In [ ]:
augmented_imgslight, augmented_gt = augment_imgs(light, uint8_imgs, gt_imgs)
augmentedlight_patches, augmentedlightgt_patches = extract_patches(augmented_imgslight, augmented_gt)
ylight = to_categorical(augmentedlightgt_patches)

## Medium augmentation

In [ ]:
augmented_imgsmedium, augmented_gt = augment_imgs(medium, uint8_imgs, gt_imgs)
augmentedmedium_patches, augmentedmediumgt_patches = extract_patches(augmented_imgsmedium, augmented_gt)
ymedium = to_categorical(augmentedmediumgt_patches)

## Strong augmentation

In [ ]:
augmented_imgsstrong, augmented_gt = augment_imgs(strong, uint8_imgs, gt_imgs)
augmentedstrong_patches, augmentedstronggt_patches = extract_patches(augmented_imgsstrong, augmented_gt)
ystrong = to_categorical(augmentedstronggt_patches)

# CNN

In [ ]:
lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.1)

## Layers

### Model 1 - Baseline

In [ ]:
model1 = models.Sequential()

model1.add(layers.Conv2D(64, (3, 3), activation='relu', padding ='same' ,input_shape=(16, 16, 3)))
model1.add(layers.MaxPooling2D((2,2)))
model1.add(layers.Conv2D(128, (3, 3), activation='relu', padding = 'same'))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(256, (3, 3), activation='relu', padding = 'same'))
model1.add(layers.Flatten())
model1.add(layers.Dense(256, activation='relu'))
model1.add(layers.Dense(1, activation='softmax'))

In [ ]:
model1.summary()

## Model 1B - sigmoid 

In [ ]:
model1B = models.Sequential()

model1B.add(layers.Conv2D(64, (3, 3), activation='relu', padding ='same' ,input_shape=(16, 16, 3)))
model1B.add(layers.MaxPooling2D((2,2)))
model1B.add(layers.Conv2D(128, (3, 3), activation='relu', padding = 'same'))
model1B.add(layers.MaxPooling2D((2, 2)))
model1B.add(layers.Conv2D(256, (3, 3), activation='relu', padding = 'same'))
model1B.add(layers.Flatten())
model1B.add(layers.Dense(256, activation='relu'))
model1B.add(layers.Dense(1, activation='sigmoid'))

## Model 1C - softmax + categorical crossentropy

In [ ]:
model1C = models.Sequential()

model1C.add(layers.Conv2D(64, (3, 3), activation='relu', padding ='same' ,input_shape=(16, 16, 3)))
model1C.add(layers.MaxPooling2D((2,2)))
model1C.add(layers.Conv2D(128, (3, 3), activation='relu', padding = 'same'))
model1C.add(layers.MaxPooling2D((2, 2)))
model1C.add(layers.Conv2D(256, (3, 3), activation='relu', padding = 'same'))
model1C.add(layers.Flatten())
model1C.add(layers.Dense(256, activation='relu'))
model1C.add(layers.Dense(2, activation='softmax'))

In [ ]:
model1C.summary()

### Model 2 - leaky relu 

In [ ]:
model2A = models.Sequential()

model2A.add(layers.Conv2D(64, (3, 3), activation=lrelu, padding ='same' ,input_shape=(16, 16, 3)))
model2A.add(layers.MaxPooling2D((3,3)))
model2A.add(layers.Conv2D(128, (3, 3), activation=lrelu, padding = 'same'))
model2A.add(layers.MaxPooling2D((3, 3)))
model2A.add(layers.Conv2D(256, (3, 3), activation=lrelu, padding = 'same'))
model2A.add(layers.Flatten())
model2A.add(layers.Dense(2, activation='softmax'))

In [ ]:
model2A.summary()

In [ ]:
model2B = models.Sequential()

model2B.add(layers.Conv2D(64, (3, 3), activation=lrelu, padding ='same' ,input_shape=(16, 16, 3)))
model2B.add(layers.MaxPooling2D((2,2)))
model2B.add(layers.Conv2D(64, (3, 3), activation=lrelu, padding = 'same'))
model2B.add(layers.MaxPooling2D((2, 2)))

model2B.add(layers.Conv2D(128, (3, 3), activation=lrelu, padding = 'same'))
model2B.add(layers.MaxPooling2D((2, 2)))
model2B.add(layers.Conv2D(128, (3, 3), activation=lrelu, padding = 'same'))
model2B.add(layers.MaxPooling2D((2, 2)))

model2B.add(layers.Conv2D(256, (3, 3), activation=lrelu, padding = 'same'))

model2B.add(layers.Flatten())
model2B.add(layers.Dense(2))
model2B.add(layers.Dense(1, activation='softmax'))

In [ ]:
model2B.summary()

In [ ]:
model2BB = models.Sequential()

model2BB.add(layers.Conv2D(64, (3, 3), activation=lrelu, padding ='same' ,input_shape=(16, 16, 3)))
model2BB.add(layers.MaxPooling2D((2,2)))
model2BB.add(layers.Conv2D(64, (3, 3), activation=lrelu, padding = 'same'))
model2BB.add(layers.MaxPooling2D((2, 2)))

model2BB.add(layers.Conv2D(128, (3, 3), activation=lrelu, padding = 'same'))
model2BB.add(layers.MaxPooling2D((2, 2)))
model2BB.add(layers.Conv2D(128, (3, 3), activation=lrelu, padding = 'same'))
model2BB.add(layers.MaxPooling2D((2, 2)))

model2BB.add(layers.Conv2D(256, (3, 3), activation=lrelu, padding = 'same'))

model2BB.add(layers.Flatten())
model2BB.add(layers.Dense(2, activation='sigmoid'))

In [ ]:
model2BB.summary()

### Model 3 - mirror padding 

In [ ]:
from keras.layers import Layer

class SymmetricPadding2D(Layer):

    def __init__(self, output_dim, padding=[1,1], 
                 data_format="channels_last", **kwargs):
        self.output_dim = output_dim
        self.data_format = data_format
        self.padding = padding
        super(SymmetricPadding2D, self).__init__(**kwargs)

    def build(self, input_shape):
        super(SymmetricPadding2D, self).build(input_shape)

    def call(self, inputs):
        if self.data_format is "channels_last":
            #(batch, depth, rows, cols, channels)
            pad = [[0,0]] + [[i,i] for i in self.padding] + [[0,0]]
        elif self.data_format is "channels_first":
            #(batch, channels, depth, rows, cols)
            pad = [[0, 0], [0, 0]] + [[i,i] for i in self.padding]

        if K.backend() == "tensorflow":
            import tensorflow as tf
            paddings = tf.constant(pad)
            out = tf.pad(inputs, paddings, "REFLECT")
        else:
            raise Exception("Backend " + K.backend() + "not implemented")
        return out 

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [ ]:
import keras
pad1 = [[2,2], [2,2], [2,2], [2,2]]
Model3_input = keras.Input(shape=(16, 16, 3))
x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(Model3_input)
x = layers.Conv2D(64,(3,3), activation = lrelu, padding = 'same')(x)
x = layers.MaxPooling2D((3,3))(x)
x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
x = layers.Conv2D(128, (3, 3), activation=lrelu, padding = 'same')(x)
x = layers.MaxPooling2D((3, 3))(x)
x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
x = layers.Conv2D(256, (3, 3), activation=lrelu, padding = 'same')(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation=lrelu)(x)
Model3_output = layers.Dense(1, activation='softmax')(x)

model3 = keras.Model(inputs=Model3_input, outputs=Model3_output, name='Model 3')


In [ ]:
model3.summary()

## Training

In [6]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [7]:
def generate_minibatch(X, Y, batch_size):
    """
    Procedure for real-time minibatch creation and image augmentation.
     This runs in a parallel thread while the model is being trained.
    """
    Y = tf.keras.utils.to_categorical(Y, 2)
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, 16,16, 3))
        Y_batch = np.empty((batch_size, 2))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X.shape[0])
            shape = X[idx].shape
                    
            # The label does not depend on the image rotation/flip (provided that the rotation is in steps of 90°)
            X_batch[i] = X[idx]
            Y_batch[i] = Y[idx]
        yield (X_batch, Y_batch)
          

In [20]:
def SoftmaxCategorical(): 
    pad1 = [[2,2], [2,2], [2,2], [2,2]]
    model_input = keras.Input(shape=(16, 16, 3))
    lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.1)
    
    #x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(model_input)
    x = layers.Conv2D(64,(5,5), activation = lrelu, padding = 'same')(model_input)
    #x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
    x = layers.MaxPooling2D((2,2))(x)

    #x = layers.Lambd#a(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
    x = layers.Conv2D(64,(3,3), activation = lrelu, padding = 'same')(x)
    #x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
    x = layers.MaxPooling2D((2,2))(x)

    #x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
    x= layers.Conv2D(128, (3, 3), activation=lrelu, padding = 'same')(x)
      
    #x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Dropout(0.2)(x)

    #x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
    x = layers.Conv2D(128, (3, 3), activation=lrelu, padding = 'same')(x)
    #x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
    x = layers.MaxPooling2D((2, 2))(x)

    #x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
    x = layers.Conv2D(256, (3, 3), activation=lrelu, padding = 'same')(x)
    #x = layers.Lambda(lambda y: tf.pad(y, paddings = pad1, mode = 'REFLECT'))(x)
    #x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(256, (3, 3), activation=lrelu, padding = 'same')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation=lrelu)(x)
    model_output = layers.Dense(2, activation='sigmoid')(x)

    model = keras.Model(inputs=model_input, outputs=model_output, name='Model 4, SoftmaxCategorical')
    
    return model 

In [21]:
import math
batch_size = 64
steps = math.ceil(img_patches.shape[0] / batch_size) 
DF = pd.DataFrame()

SoftC = SoftmaxCategorical()
SoftC.summary()
SoftC.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3))
Hist = SoftC.fit_generator(generate_minibatch(img_patches,gt_patches.reshape(-1), batch_size), epochs=1, steps_per_epoch= steps)
#Hist = SoftB.fit(imgsall_patches,gtall_patches,  epochs=10, batch_size = 32)

DF.append(Hist.history['val_accuracy'])
        
predictions = SoftC.predict(img_patches_test)
predictions = np.squeeze(helper.binarize_predictions(predictions))
predictions = predictions[:,1]
sub.create_pred_images(predictions)
submission_filename = 'model4_SoftB_all.csv'
sub.create_submission(submission_filename)

Model: "Model 4, SoftmaxCategorical"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 16, 16, 3)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 16, 16, 64)        4864      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 2, 

KeyError: 'val_accuracy'

### Model 1 - original data - augmented data 

#### Softmax + binary crossentropy 

In [ ]:
#ORIGINAL DATA 
model1.compile(loss = 'binary_crossentropy',  optimizer = 'adam', metrics = ['accuracy', f1_m])
M1O = model1.fit(img_patches, gt_patches, batch_size = 32, epochs = 2, validation_split = 0.2)

In [ ]:
#LIGHT AUGMENTATION
model1.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy', f1_m])
M1L = model1.fit(augmentedlight_patches, augmentedlightgt_patches, batch_size = 32, epochs = 2, validation_split = 0.2)

In [ ]:
#print(M1O.history['val_f1_m'])

In [ ]:
#MEDIUM AUGMENTATIONS
M1M = model1.fit(augmentedmedium_patches, augmentedmediumgt_patches, batch_size = 32, epochs = 2, validation_split = 0.2)

In [ ]:
#STRONG AUGMENTATION
M1S = model1.fit(augmentedstrong_patches, augmentedstronggt_patches, batch_size = 32, epochs = 2, validation_split = 0.2)

#### sigmoid + binary crossentropy

In [ ]:
#ORIGINAL DATA 
model1B.compile(loss = 'binary_crossentropy',  optimizer = 'adam', metrics = ['accuracy', f1_m])
M1BO = model1B.fit(img_patches, gt_patches, batch_size = 32, epochs = 5, validation_split = 0.2)

In [ ]:
#LIGHT AUGMENTATION 
M1BL = model1B.fit(augmentedlight_patches, augmentedlightgt_patches, batch_size = 32, epochs = 5, validation_split = 0.2)

In [ ]:
M1BM = model1B.fit(augmentedmedium_patches, augmentedmediumgt_patches, batch_size = 32, epochs = 5, validation_split = 0.2)

In [ ]:
M1BS = model1B.fit(augmentedstrong_patches, augmentedstronggt_patches, batch_size = 32, epochs = 5, validation_split = 0.2)

#### Softmax + categorical crossentropy 

In [ ]:
#ORIGINAL DATA 
model1C.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
M1CO = model1C.fit(img_patches, y, batch_size = 32, epochs = 4, validation_split = 0.2)

In [ ]:
#LIGHT AUGMENTATION
model1C.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
M1LC = model1C.fit(augmentedlight_patches, ylight, batch_size = 32, epochs = 2, validation_split = 0.2)

### PLOTTING MODEL 1 A-B-C

In [ ]:
model1_df = pd.DataFrame()

In [ ]:
Results = pd.DataFrame(columns = ['Model', 'Data','F1 score', 'Epochs'])

Results['F1 score'] = M1O.history['val_f1_m']
Results['Model'] = 1 
Results['Data'] = 'Original'
Results['Epochs'] = np.arange(1,len(M1O.history['val_f1_m']))

## Model 2 - leaky relu

### A : softmax + binary 

In [ ]:
#original data 
model2A.compile(loss = 'binary_crossentropy',  optimizer = 'adam', metrics = ['accuracy', f1_m])
M2AO = model2A.fit(img_patches, gt_patches, batch_size = 32, epochs = 4, validation_split = 0.2)

In [ ]:
#augmented light
M2AL = model2A.fit(augmentedlight_patches, augmentedlightgt_patches, batch_size = 32, epochs = 4, validation_split = 0.2)

In [ ]:
#augmented medium 
M2AM = model2A.fit(augmentedmedium_patches, augmentedmediumgt_patches, batch_size = 32, epochs = 4, validation_split = 0.2)

In [ ]:
M2AS = model2A.fit(augmentedstrong_patches, augmentedstronggt_patches, batch_size = 32, epochs = 4, validation_split = 0.2)

### AA: A with softmax + categorical

In [ ]:
model2A.compile(loss = 'categorical_crossentropy',  optimizer = 'adam', metrics = ['accuracy'])
M2AAO = model2A.fit(img_patches, y, batch_size = 32, epochs = 4, validation_split = 0.2)

### B: softmax + binary (Cindy model)

In [ ]:
model2B.compile(loss = 'binary_crossentropy',  optimizer = 'adam', metrics = ['accuracy', f1_m])
M2BO = model2B.fit(img_patches, gt_patches, batch_size = 32, epochs = 4, validation_split = 0.2)

In [ ]:
M2BL = model2B.fit(augmentedlight_patches, augmentedlightgt_patches, batch_size = 32, epochs = 4, validation_split = 0.2)

In [ ]:
M2BM = model2B.fit(augmentedmedium_patches, augmentedmediumgt_patches, batch_size = 32, epochs = 4, validation_split = 0.2)

In [ ]:
M2BS = model2B.fit(augmentedstrong_patches, augmentedstronggt_patches, batch_size = 32, epochs = 4, validation_split = 0.2)

### sigmoid + categorical

In [ ]:
model2BB.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
M2BBO = model2BB.fit(img_patches, y, batch_size = 32, epochs = 4, validation_split = 0.2)

In [ ]:
M2BBL = model2BB.fit(augmentedlight_patches, ylight, batch_size = 32, epochs = 4, validation_split = 0.2)

In [ ]:
print(img_patches.shape)

In [ ]:
import math
import keras
batch_size = 64
steps_per_epoch = math.ceil(img_patches.shape[0] / batch_size)
"""def softmax_categorical_crossentropy(y_true, y_pred):
       
        Uses categorical cross-entropy from logits in order to improve numerical stability.
        This is especially useful for TensorFlow (less useful for Theano).           
        return K.categorical_crossentropy(y_pred, y_true, from_logits=True)
    """
model2BB.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
#model.fit(img_patches, gt_patches, batch_size = 32, epochs = 2, validation_split = 0.2)
#F1 = 0.645	secondary = 0.805
model2BB.fit_generator(generate_minibatch(img_patches,gt_patches, batch_size),
                    epochs=10, steps_per_epoch= steps_per_epoch)

In [ ]:
full = imgs.copy()
fullgt = gt_imgs.copy()

for n in range(len(fullgt)):
        augmented = helper.image_augmentation(image_size = 256)(image=full[n], mask=fullgt[n])
        full.extend(augmented['image'])
        fullgt.extend(augmented['mask'])
        
full_patches, fullgt_patches = extract_patches(full, fullgt)

In [ ]:
model2BB.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
# F1 = 0.638	0.800
M2BBfullgen = model2BB.fit_generator(generate_minibatch(full_patches,fullgt_patches, batch_size),
                    epochs=10, steps_per_epoch= steps_per_epoch)

In [ ]:
complete = imgs.copy()
completegt = gt_imgs.copy()

for n in range(len(completegt)):
        augmented = helper.pre_process()(image=complete[n], mask=completegt[n])
        complete.extend(augmented['image'])
        completegt.extend(augmented['mask'])
        
complete_patches, completegt_patches = extract_patches(complete, completegt)

In [ ]:
modelFIN = models.Sequential()

modelFIN.add(layers.Conv2D(64, (3, 3), activation=lrelu, padding ='same' ,input_shape=(16, 16, 3)))
modelFIN.add(layers.MaxPooling2D((2,2)))
modelFIN.add(layers.Conv2D(64, (3, 3), activation=lrelu, padding = 'same'))
modelFIN.add(layers.MaxPooling2D((2, 2)))

modelFIN.add(layers.Conv2D(128, (3, 3), activation=lrelu, padding = 'same'))
modelFIN.add(layers.MaxPooling2D((2, 2)))
modelFIN.add(layers.Conv2D(128, (3, 3), activation=lrelu, padding = 'same'))
modelFIN.add(layers.MaxPooling2D((2, 2)))

modelFIN.add(layers.Conv2D(256, (3, 3), activation=lrelu, padding = 'same'))

modelFIN.add(layers.Flatten())
modelFIN.add(layers.Dense(2, activation='sigmoid'))

In [ ]:
uint8_imgsfull = [None] * len(full)
for i, image in enumerate(full): 
    uint8_imgfull = helper.img_float_to_uint8(image)
    uint8_imgsfull[i] = uint8_imgfull

In [ ]:
augmented_imgslightfull, augmented_gtfull = augment_imgs(light, uint8_imgsfull, fullgt)
augmentedlight_patchesfull, augmentedlightgt_patchesfull = extract_patches(augmented_imgslightfull, augmented_gtfull)
ylightfull = to_categorical(augmentedlightgt_patchesfull)

In [ ]:
modelFIN.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
M2BBfullgenL = modelFIN.fit_generator(generate_minibatch(augmentedlight_patchesfull, augmentedlightgt_patchesfull, batch_size),
                    epochs=10, steps_per_epoch= steps_per_epoch)

In [ ]:
albu.augmentations.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0)

In [ ]:
model2BB.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
M2BBLgen = model2BB.fit_generator(generate_minibatch(augmentedlight_patches,augmentedlightgt_patches, batch_size),
                    epochs=8, steps_per_epoch= steps_per_epoch)

In [ ]:
M2BBMgen = model2BB.fit_generator(generate_minibatch(augmentedmedium_patches,augmentedmediumgt_patches, batch_size),
                    epochs=8, steps_per_epoch= steps_per_epoch)

## Model 3 - original data - augmented data 

In [ ]:
model3.compile(loss = 'binary_crossentropy',  optimizer = 'adam', metrics = ['accuracy', f1_m])
M3O = model3.fit_generator(img_patches, gt_patches, batch_size = 15, epochs = 4, validation_split = 0.2)

In [ ]:
model3.compile(loss = 'binary_crossentropy',  optimizer = 'adam', metrics = ['accuracy', f1_m])
M3L = model3.fit(augmentedlight_patches, augmentedlightgt_pathes, batch_size = 32, epochs = 4, validation_split = 0.2)

## Predictions

In [ ]:
predictions = model.predict(img_patches)

In [ ]:
predictions = np.squeeze(helper.binarize_predictions(predictions))

In [ ]:
predictions = predictions[:,1]

In [ ]:
predictions

### Displaying predictions

In [ ]:
# Display prediction as an image
img_idx = 49
img_indexed = imgs[img_idx]
img_pat = helper.img_crop(img_indexed, patch_size, patch_size)
img_pat = np.asarray(img_pat)
prediction = model.predict(img_pat)

prediction = prediction[:,1]

w = gt_imgs[img_idx].shape[0]
h = gt_imgs[img_idx].shape[1]

prediction = helper.binarize_predictions(prediction)
prediction = single_patch_cleaner(prediction.reshape((w//patch_size, h//patch_size))).flatten()

predicted_im = helper.label_to_img(w, h, patch_size, patch_size, prediction)

cimg = helper.concatenate_images(imgs[img_idx], predicted_im)
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 
plt.imshow(cimg, cmap='Greys_r')

new_img = helper.make_img_overlay(imgs[img_idx], predicted_im)

plt.imshow(new_img)

# Create Submission

In [ ]:
import submission as sub

In [ ]:
n = 50
patch_size = 16

img_test = []

for i in range(1, 51):
    image_filename = 'Datasets/test_set_images/test_' + str(i) + '/test_' + str(i) + '.png' 
    img_test.append(helper.load_image(image_filename))

img_patches_test = [helper.img_crop(img_test[i], patch_size, patch_size) for i in range(n)]
img_patches_test = np.asarray([img_patches_test[i][j] for i in range(len(img_patches_test)) for j in range(len(img_patches_test[i]))])

In [ ]:
predictions = model2BB.predict(img_patches_test)
predictions = np.squeeze(helper.binarize_predictions(predictions))

In [ ]:
predictions = predictions[:,1]

In [ ]:
sub.create_pred_images(predictions)

In [ ]:
submission_filename = 'submission2BBfullgen.csv'
   
sub.create_submission(submission_filename)

### Displaying predictions

In [ ]:
# Display prediction as an image
img_idx = 49
img_indexed = img_test[img_idx]
img_pat = helper.img_crop(img_indexed, patch_size, patch_size)
img_pat = np.asarray(img_pat)
prediction = model.predict(img_pat)

prediction = prediction[:,1]

w = img_test[img_idx].shape[0]
h = img_test[img_idx].shape[1]

prediction = helper.binarize_predictions(prediction)
prediction = single_patch_cleaner(prediction.reshape((w//patch_size, h//patch_size)))

predicted_im = helper.label_to_img(w, h, patch_size, patch_size, prediction.flatten())
cimg = helper.concatenate_images(img_test[img_idx], predicted_im)
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 
plt.imshow(cimg, cmap='Greys_r')

new_img = helper.make_img_overlay(img_test[img_idx], predicted_im)

plt.imshow(new_img)